In [1]:
import pandas as pd
import mysql.connector

In [2]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_45'     
)
cursor = connection.cursor()

### Bài 1

In [3]:
# Khóa chính của bảng record_id

### Bài 2

In [4]:
# Bảng đã đạt 1NF
# Bảng có các thuộc tính khác nhau phụ thuốc vào các khóa khác nhau như item_code, warehouse_id, customer_id, staff_id nên không thể đạt 2NF
# Các thuộc tính vi phạm 3NF: item_name, quantity, warehouse_name, location, customer_name, staff_name

### Bài 3

In [5]:
cursor.execute("""
CREATE TABLE Items (
    item_code VARCHAR(5) PRIMARY KEY,
    item_name VARCHAR(100) NOT NULL
);
""")

In [6]:
cursor.execute("""
INSERT INTO Items (item_code, item_name) VALUES
('IT001', 'Motor Oil 5L'),
('IT002', 'Engine Cleaner'),
('IT003', 'Battery 12V');
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE Warehouse (
    warehouse_id VARCHAR(4) PRIMARY KEY,
    warehouse_name VARCHAR(100) NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO Warehouse (warehouse_id, warehouse_name) VALUES
('WH01', 'Main Warehouse'),
('WH02', 'North Depot');
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE Customers (
    customer_id VARCHAR(4) PRIMARY KEY,
    customer_name VARCHAR(100) NOT NULL
);
""")

In [10]:
cursor.execute("""
INSERT INTO Customers (customer_id, customer_name) VALUES
('C001', 'Mekong Corp'),
('C002', 'Vina Auto'),
('C003', 'Delta Co');
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE Staffs (
    staff_id VARCHAR(3) PRIMARY KEY,
    staff_name VARCHAR(100) NOT NULL
);
""")

In [12]:
cursor.execute("""
INSERT INTO Staffs (staff_id, staff_name) VALUES
('S01', 'Hoa Nguyen'),
('S02', 'Khang Tran');
""")

connection.commit()

In [13]:
cursor.execute("""
CREATE TABLE RecordDetails (
    record_id VARCHAR(4) PRIMARY KEY,
    item_code VARCHAR(5) NOT NULL,
    warehouse_id VARCHAR(4) NOT NULL,
    customer_id VARCHAR(4) NOT NULL,
    staff_id VARCHAR(3) NOT NULL,
    quantity INT NOT NULL,
    location VARCHAR(100) NOT NULL, 
    FOREIGN KEY (item_code) REFERENCES Items(item_code),
    FOREIGN KEY (warehouse_id) REFERENCES Warehouse(warehouse_id),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY (staff_id) REFERENCES Staffs(staff_id)
);
""")

In [14]:
cursor.execute("""
INSERT INTO RecordDetails (record_id, item_code, warehouse_id, customer_id, staff_id, quantity, location) VALUES
('R001', 'IT001', 'WH01', 'C001', 'S01', 50, 'Shelf A1'),
('R002', 'IT002', 'WH01', 'C002', 'S02', 30, 'Shelf B2'),
('R003', 'IT001', 'WH02', 'C001', 'S01', 40, 'Shelf C3'),
('R004', 'IT003', 'WH01', 'C003', 'S02', 25, 'Shelf A1');
""")

connection.commit()

### Bài 4

In [24]:
query = ("""
SELECT c.warehouse_name
    , b.item_name
    , SUM(a.quantity) Total_Quantity
 FROM RecordDetails a
LEFT JOIN Items b on a.item_code = b.item_code
LEFT JOIN Warehouse c on a.warehouse_id = c.warehouse_id
GROUP BY c.warehouse_name, b.item_name
""")

data_4a = pd.read_sql_query(query, connection)
print(data_4a)

   warehouse_name       item_name  Total_Quantity
0  Main Warehouse    Motor Oil 5L            50.0
1  Main Warehouse  Engine Cleaner            30.0
2     North Depot    Motor Oil 5L            40.0
3  Main Warehouse     Battery 12V            25.0


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_8764\1717026112.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4a = pd.read_sql_query(query, connection)


In [29]:
query = ("""
SELECT b.customer_name
    , c.item_name
 FROM RecordDetails a
LEFT JOIN Customers b on a.customer_id = b.customer_id
LEFT JOIN Items c on a.item_code = c.item_code
GROUP BY b.customer_name, c.item_name
""")

data_4b = pd.read_sql_query(query, connection)
print(data_4b)

  customer_name       item_name
0   Mekong Corp    Motor Oil 5L
1     Vina Auto  Engine Cleaner
2      Delta Co     Battery 12V


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_8764\3768883129.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4b = pd.read_sql_query(query, connection)


In [31]:
query = ("""
SELECT b.staff_name
    , c.item_name
 FROM RecordDetails a
LEFT JOIN Staffs b on a.staff_id = b.staff_id
LEFT JOIN Items c on a.item_code = c.item_code
WHERE a.warehouse_id in (SELECT warehouse_id FROM Warehouse WHERE warehouse_name = 'Main Warehouse')
""")

data_4c = pd.read_sql_query(query, connection)
print(data_4c)

   staff_name       item_name
0  Hoa Nguyen    Motor Oil 5L
1  Khang Tran  Engine Cleaner
2  Khang Tran     Battery 12V


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_8764\4012103865.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4c = pd.read_sql_query(query, connection)


In [35]:
query = ("""
SELECT b.item_name
    , a.location
 FROM RecordDetails a
LEFT JOIN Items b on a.item_code = b.item_code
GROUP BY b.item_name, a.location
""")

data_4d = pd.read_sql_query(query, connection)
print(data_4d)

        item_name  location
0    Motor Oil 5L  Shelf A1
1  Engine Cleaner  Shelf B2
2    Motor Oil 5L  Shelf C3
3     Battery 12V  Shelf A1


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_8764\2889416905.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4d = pd.read_sql_query(query, connection)


In [44]:
query = ("""
SELECT b.warehouse_name
    , COUNT(DISTINCT (a.customer_id)) Total_Customer
 FROM RecordDetails a
LEFT JOIN Warehouse b on a.warehouse_id = b.warehouse_id
GROUP BY b.warehouse_name
""")

data_4d = pd.read_sql_query(query, connection)
print(data_4d)

   warehouse_name  Total_Customer
0  Main Warehouse               3
1     North Depot               1


C:\Users\Apollonir\AppData\Local\Temp\ipykernel_8764\268266142.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_4d = pd.read_sql_query(query, connection)
